# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [5]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
from torch.autograd import Variable

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [6]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [7]:
class SimpleNet(nn.Module):
# TODO:define model
    #A three-layer fully connected linear network is established with the nonlinear function ReLU and the batch standardization
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(in_dim,n_hidden_1),nn.BatchNorm1d(n_hidden_1),nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(n_hidden_1,n_hidden_2),nn.BatchNorm1d(n_hidden_2),nn.ReLU(True))
        self.layer3 = nn.Sequential(nn.Linear(n_hidden_2,out_dim))
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    
model = SimpleNet(28*28, 300, 100 ,10)

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

Next, we can start to train and evaluate!

In [8]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_accuracy = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        images = Variable(images.view(images.size(0),-1))
        labels = Variable(labels)
        
        out = model(images)
        loss = criterion(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    
        
# evaluate
# TODO:calculate the accuracy using traning and testing dataset
model.eval()
train_accuracy = 0
test_accuracy = 0

for data in train_loader:
    images, labels = data
    images = images.view(images.size(0), -1)
    
    #calculate the training accuracy
    out = model(images)    
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    train_accuracy += num_correct.item() 

for data in test_loader:
    images, labels = data
    images = images.view(images.size(0), -1)

    #predict and calculate the testing accuracy
    out = model(images)  
    _, pred = torch.max(out, 1)
    num_correct = (pred == labels).sum()
    test_accuracy += num_correct.item() 

100%|████████████████████████████████████████████████████████████████████████████████| 468/468 [00:16<00:00, 28.87it/s]


#### Q5:
Please print the training and testing accuracy.

In [9]:
#predict the training accuracy and testing accuracy
print('Training Accuracy: {:.2f}%, Testing Accuracy: {:.2f}%'.format(train_accuracy*100 / (len(train_dataset)),test_accuracy*100 / (len(test_dataset))))

Training Accuracy: 98.88%, Testing Accuracy: 97.60%
